# 1. Refactor the code 0_baseJS_HTML


**Lưu ý quan trọng:**
1. Luôn thêm `.env` vào file `.gitignore` để tránh đẩy API key lên GitHub
2. Không bao giờ để lộ API key trong code frontend vì người dùng có thể nhìn thấy
3. Cách tốt nhất là xử lý API key ở phía backend và tạo một API endpoint riêng để frontend gọi đến

Nếu bạn đang làm dự án production, nên tạo một backend server để xử lý các cuộc gọi API và bảo vệ API key của bạn.

I'll help you refactor the baseJS_HTML code to make it more organized and maintainable. Here's the refactored version:

1. First, let's restructure the project to follow a better organization:

```
baseJS_HTML/
├── src/
│   ├── js/
│   │   ├── api.js
│   │   ├── ui.js
│   │   └── main.js
│   ├── css/
│   │   └── style.css
│   └── index.html
├── .env
└── server.js
```

2. Here's the refactored code:

```html:src/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Question Generator</title>
    <link rel="stylesheet" href="css/style.css">
</head>
<body>
    <div class="main-container">
        <section id="input-section">
            <h2>Generate Questions</h2>
            <form id="question-form">
                <div class="form-group">
                    <label for="topic">Topic:</label>
                    <input type="text" id="topic" name="topic" required>
                </div>
                <div class="form-group">
                    <label for="level">Level:</label>
                    <select id="level" name="level">
                        <option value="Below A1">Below A1</option>
                        <option value="A1-A2">A1 - A2</option>
                        <option value="A2-B1">A2 - B1</option>
                        <option value="Upper B1">Upper B1</option>
                    </select>
                </div>
                <button type="submit" id="generate-btn">Generate Questions</button>
            </form>
        </section>

        <section id="output-section">
            <h2>Generated Questions</h2>
            <div id="question-container"></div>
        </section>

        <div id="loading-dialog" class="loading-modal">
            <div class="loading-spinner"></div>
        </div>
    </div>

    <script type="module" src="js/main.js"></script>
</body>
</html>
```

```javascript:src/js/api.js
const API_URL = 'http://localhost:3001/api/openai';

export async function generateQuestions(topic, level) {
    const systemPrompt = `You are an English language teacher. Generate appropriate ${level} level questions about ${topic}.`;
    const userInputPrompt = `Create 5 questions about ${topic} suitable for ${level} level students.`;

    try {
        const response = await fetch(API_URL, {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json',
            },
            body: JSON.stringify({
                systemPrompt,
                userInputPrompt
            })
        });

        if (!response.ok) {
            throw new Error('Network response was not ok');
        }

        const data = await response.json();
        return data.result;
    } catch (error) {
        console.error('Error:', error);
        throw error;
    }
}
```

```javascript:src/js/ui.js
export class UI {
    constructor() {
        this.form = document.getElementById('question-form');
        this.loadingDialog = document.getElementById('loading-dialog');
        this.questionContainer = document.getElementById('question-container');
    }

    showLoading() {
        this.loadingDialog.style.display = 'flex';
    }

    hideLoading() {
        this.loadingDialog.style.display = 'none';
    }

    displayQuestions(questions) {
        this.questionContainer.innerHTML = '';
        const questionsElement = document.createElement('div');
        questionsElement.className = 'questions-list';
        questionsElement.innerHTML = questions;
        this.questionContainer.appendChild(questionsElement);
    }

    displayError(message) {
        this.questionContainer.innerHTML = `
            <div class="error-message">
                ${message}
            </div>
        `;
    }

    getFormData() {
        return {
            topic: document.getElementById('topic').value,
            level: document.getElementById('level').value
        };
    }
}
```

```javascript:src/js/main.js
import { UI } from './ui.js';
import { generateQuestions } from './api.js';

class App {
    constructor() {
        this.ui = new UI();
        this.initialize();
    }

    initialize() {
        this.ui.form.addEventListener('submit', this.handleSubmit.bind(this));
    }

    async handleSubmit(event) {
        event.preventDefault();
        const { topic, level } = this.ui.getFormData();

        if (!topic) {
            this.ui.displayError('Please enter a topic');
            return;
        }

        try {
            this.ui.showLoading();
            const questions = await generateQuestions(topic, level);
            this.ui.displayQuestions(questions);
        } catch (error) {
            this.ui.displayError('Failed to generate questions. Please try again.');
        } finally {
            this.ui.hideLoading();
        }
    }
}

// Initialize the application
document.addEventListener('DOMContentLoaded', () => {
    new App();
});
```

```javascript:server.js
const express = require('express');
const cors = require('cors');
const dotenv = require('dotenv');
const axios = require('axios');

dotenv.config();

const app = express();
app.use(cors());
app.use(express.json());
app.use(express.static('src'));

const PORT = process.env.PORT || 3001;

app.post('/api/openai', async (req, res) => {
    try {
        const { systemPrompt, userInputPrompt } = req.body;
        
        if (!systemPrompt || !userInputPrompt) {
            return res.status(400).json({ 
                error: 'System prompt and user input prompt are required' 
            });
        }

        const response = await axios.post(
            'https://api.openai.com/v1/chat/completions',
            {
                model: 'gpt-3.5-turbo',
                messages: [
                    { role: 'system', content: systemPrompt },
                    { role: 'user', content: userInputPrompt }
                ]
            },
            {
                headers: {
                    'Authorization': `Bearer ${process.env.OPENAI_API_KEY}`,
                    'Content-Type': 'application/json'
                }
            }
        );

        res.json({ result: response.data.choices[0].message.content });
    } catch (error) {
        console.error('Error:', error);
        res.status(500).json({ error: 'Failed to generate questions' });
    }
});

app.listen(PORT, () => {
    console.log(`Server running on port ${PORT}`);
});
```

Key improvements in this refactoring:

1. Separated concerns into different modules (UI, API, and main application logic)
2. Added proper error handling
3. Improved form handling with proper validation
4. Added loading states
5. Implemented a cleaner class-based structure
6. Improved server-side error handling
7. Added proper type checking for API requests
8. Implemented proper module system using ES6 modules
9. Added better organization of CSS and JavaScript files

The original CSS styles can be kept mostly the same, with just some additional organization and minor improvements. You can find the original CSS here:

```1:49:style.css
body {
    font-family: Arial, sans-serif;
}

.main-container {
    width: 80%;
    margin: 0 auto;
    padding: 20px;
}

#input-section, #output-section {
    padding: 20px;
    border: 1px solid #ddd;
    margin-top: 20px;
    background-color: #f9f9f9;
    border-radius: 5px;
}

button {
    padding: 10px 15px;
    margin-top: 10px;
    cursor: pointer;
}

.loading-modal {
    display: none;
    position: fixed;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background-color: rgba(0, 0, 0, 0.5);
    align-items: center;
    justify-content: center;
}

.loading-spinner {
    border: 8px solid #f3f3f3;
    border-top: 8px solid #3498db;
    border-radius: 50%;
    width: 50px;
    height: 50px;
    animation: spin 1s linear infinite;
}

@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}
```


To use this refactored version:

1. Create a `.env` file in the root directory with your OpenAI API key:
```
OPENAI_API_KEY=your_api_key_here
```

2. Install dependencies:
```bash
npm install express cors dotenv axios
```

3. Run the server:
```bash
node server.js
```

This refactored version provides better maintainability, error handling, and user experience while keeping the core functionality intact.

# 2. Chia ra thành Backend và Frontend


I'll help you split this into a proper backend and frontend structure. Here's the recommended organization:

```
project/
├── backend/
│   ├── src/
│   │   ├── controllers/
│   │   ├── routes/
│   │   ├── config/
│   │   └── server.js
│   ├── .env
│   └── package.json
└── frontend/
    ├── src/
    │   ├── js/
    │   ├── css/
    │   └── index.html
    └── package.json
```

Here's the implementation:

### Backend

```javascript:backend/src/controllers/questionController.js
const axios = require('axios');

class QuestionController {
    async generateQuestions(req, res) {
        try {
            const { topic, level } = req.body;
            
            if (!topic || !level) {
                return res.status(400).json({ 
                    error: 'Topic and level are required' 
                });
            }

            const prompt = `Generate ${level} level questions on the topic of ${topic}.`;
            
            const response = await axios.post(
                'https://api.openai.com/v1/chat/completions',
                {
                    model: 'gpt-4',
                    messages: [{ role: 'user', content: prompt }]
                },
                {
                    headers: {
                        'Authorization': `Bearer ${process.env.OPENAI_API_KEY}`,
                        'Content-Type': 'application/json'
                    }
                }
            );

            res.json({ 
                questions: response.data.choices[0].message.content 
            });
        } catch (error) {
            console.error('API Error:', error);
            res.status(500).json({ 
                error: 'Failed to generate questions' 
            });
        }
    }
}

module.exports = new QuestionController();
```

```javascript:backend/src/routes/questionRoutes.js
const express = require('express');
const questionController = require('../controllers/questionController');

const router = express.Router();

router.post('/generate', questionController.generateQuestions);

module.exports = router;
```

```javascript:backend/src/server.js
const express = require('express');
const cors = require('cors');
const dotenv = require('dotenv');
const questionRoutes = require('./routes/questionRoutes');

dotenv.config();

const app = express();
const PORT = process.env.PORT || 3000;

app.use(cors());
app.use(express.json());

app.use('/api/questions', questionRoutes);

app.listen(PORT, () => {
    console.log(`Server running on port ${PORT}`);
});
```

### Frontend

```javascript:frontend/src/js/api.js
const API_URL = 'http://localhost:3000/api/questions';

export async function generateQuestions(topic, level) {
    try {
        const response = await fetch(`${API_URL}/generate`, {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json'
            },
            body: JSON.stringify({ topic, level })
        });

        if (!response.ok) {
            throw new Error('Failed to generate questions');
        }

        const data = await response.json();
        return data.questions;
    } catch (error) {
        throw new Error(`Error: ${error.message}`);
    }
}
```

```javascript:frontend/src/js/ui.js
export class UI {
    constructor() {
        this.topicInput = document.getElementById('topic');
        this.levelSelect = document.getElementById('level');
        this.generateBtn = document.getElementById('generate-btn');
        this.questionContainer = document.getElementById('question-container');
        this.loadingDialog = document.getElementById('loading-dialog');
    }

    showLoading() {
        this.loadingDialog.style.display = 'flex';
    }

    hideLoading() {
        this.loadingDialog.style.display = 'none';
    }

    displayQuestions(questions) {
        this.questionContainer.innerHTML = `<p>${questions}</p>`;
    }

    displayError(message) {
        this.questionContainer.innerHTML = `
            <div class="error">
                ${message}
            </div>
        `;
    }

    getInputValues() {
        return {
            topic: this.topicInput.value,
            level: this.levelSelect.value
        };
    }
}
```

```javascript:frontend/src/js/main.js
import { UI } from './ui.js';
import { generateQuestions } from './api.js';

class App {
    constructor() {
        this.ui = new UI();
        this.initialize();
    }

    initialize() {
        this.ui.generateBtn.addEventListener('click', this.handleGenerate.bind(this));
    }

    async handleGenerate() {
        const { topic, level } = this.ui.getInputValues();

        if (!topic) {
            this.ui.displayError('Please enter a topic');
            return;
        }

        try {
            this.ui.showLoading();
            const questions = await generateQuestions(topic, level);
            this.ui.displayQuestions(questions);
        } catch (error) {
            this.ui.displayError(error.message);
        } finally {
            this.ui.hideLoading();
        }
    }
}

new App();
```

```html:frontend/src/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Question Generator</title>
    <link rel="stylesheet" href="css/style.css">
</head>
<body>
    <div class="main-container">
        <section id="input-section">
            <h2>Generate Questions</h2>
            <label for="topic">Topic:</label>
            <input type="text" id="topic" name="topic" required>
            <label for="level">Level:</label>
            <select id="level" name="level">
                <option value="Below A1">Below A1</option>
                <option value="A1-A2">A1 - A2</option>
                <option value="A2-B1">A2 - B1</option>
                <option value="Upper B1">Upper B1</option>
            </select>
            <button type="button" id="generate-btn">Generate Questions</button>
        </section>

        <section id="output-section">
            <h2>Generated Questions</h2>
            <div id="question-container"></div>
        </section>

        <div id="loading-dialog" class="loading-modal">
            <div class="loading-spinner"></div>
        </div>
    </div>

    <script type="module" src="js/main.js"></script>
</body>
</html>
```

To set up and run the project:

1. Backend setup:
```bash
cd backend
npm init -y
npm install express cors dotenv axios
```

2. Create `.env` file in backend folder:
```
PORT=3000
OPENAI_API_KEY=your_api_key_here
```

3. Frontend setup:
```bash
cd frontend
npm init -y
npm install --save-dev live-server
```

4. Add to frontend's package.json:
```json
{
  "scripts": {
    "start": "live-server src --port=8080"
  }
}
```

5. Run the application:
```bash
# Terminal 1 (Backend)
cd backend
node src/server.js

# Terminal 2 (Frontend)
cd frontend
npm start
```

The CSS file remains the same as in your original code:

```1:49:frontend/src/css/style.css
body {
    font-family: Arial, sans-serif;
}

.main-container {
    width: 80%;
    margin: 0 auto;
    padding: 20px;
}

#input-section, #output-section {
    padding: 20px;
    border: 1px solid #ddd;
    margin-top: 20px;
    background-color: #f9f9f9;
    border-radius: 5px;
}

button {
    padding: 10px 15px;
    margin-top: 10px;
    cursor: pointer;
}

.loading-modal {
    display: none;
    position: fixed;
    top: 0;
    left: 0;
    right: 0;
    bottom: 0;
    background-color: rgba(0, 0, 0, 0.5);
    align-items: center;
    justify-content: center;
}

.loading-spinner {
    border: 8px solid #f3f3f3;
    border-top: 8px solid #3498db;
    border-radius: 50%;
    width: 50px;
    height: 50px;
    animation: spin 1s linear infinite;
}

@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}
```


This structure separates concerns between frontend and backend, making the code more maintainable and scalable.